In [ ]:
from ruamel.yaml import YAML
import numpy as np
import xarray as xr
from pathlib import Path
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import matplotlib.patches as mpatches
import seaborn as sns

strength_cmap = sns.cubehelix_palette(start=0.5, rot=-0.5, as_cmap=True)

from sdm_eurec4a.visulization import (
    set_custom_rcParams,
    set_paper_rcParams,
    label_from_attrs,
    adjust_lightness_array,
    plot_one_one,
    handler_map_alpha,
    save_figure,
    add_subplotlabel,
)
from sdm_eurec4a import RepositoryPath
from sdm_eurec4a import data_loading
from sdm_eurec4a.reductions import mean_and_stderror_of_mean
from sdm_eurec4a.conversions import (
    msd_from_psd_dataarray,
    potential_temperature_from_temperature_pressure,
    relative_humidity_from_tps,
    temperature_from_potential_temperature_pressure,
)
from sdm_eurec4a.input_processing import models as smodels
from sdm_eurec4a.identifications import match_clouds_and_cloudcomposite, match_clouds_and_dropsondes
from sdm_eurec4a.constants import TimeSlices


default_colors = set_paper_rcParams()
default_dark_colors = adjust_lightness_array(default_colors, 0.75)

RepoPaths = RepositoryPath("levante")

OBS_data_dir = RepoPaths.data_dir
input_data_dir = OBS_data_dir / Path("model/input_v4.2")

CLEO_data_dir = RepoPaths.CLEO_data_dir / Path("output_v4.4-CLEO_v0.39.7-input_v4.2")
CLEO_data_dir_v43 = RepoPaths.CLEO_data_dir / Path("output_v4.3-CLEO_v0.39.7-input_v4.2")
CLEO_data_dir_v42 = RepoPaths.CLEO_data_dir / Path("output_v4.2")
CLEO_data_dir_v41 = RepoPaths.CLEO_data_dir / Path("output_v4.1")
CLEO_data_dir_v40 = RepoPaths.CLEO_data_dir / Path("output_v4.0")


fig_dir = RepoPaths.fig_dir / Path("paper-v4.4")
fig_dir.mkdir(exist_ok=True, parents=False)
appendix_fig_dir = fig_dir / Path("appendix")
appendix_fig_dir.mkdir(exist_ok=True, parents=False)

In [ ]:
small_fig_size = np.array((16 / 3, 9 / 3))
square_fig_size = small_fig_size[0], small_fig_size[0]
large_figure_multiplicator = 12 / 8.3
large_fig_size = small_fig_size * large_figure_multiplicator
large_square_fig_size = large_fig_size[0], large_fig_size[0]
wide_fig_size = large_fig_size[0], small_fig_size[1]

<!-- ## Data loading -->

In [ ]:
cloud_composite = xr.open_dataset(
    OBS_data_dir / Path("observation/cloud_composite/processed/cloud_composite_SI_units_20241025.nc"),
)
dropsonde = xr.open_dataset(
    OBS_data_dir / Path("observation/dropsonde/processed/drop_sondes.nc"),
)
ds_distances = xr.open_dataset(
    OBS_data_dir
    / Path("observation/combined/distance/distance_dropsondes_identified_clusters_rain_mask_5.nc"),
)

identified_clusters = xr.open_dataset(
    OBS_data_dir
    / Path(
        "observation/cloud_composite/processed/identified_clusters/identified_clusters_rain_mask_5.nc"
    )
)
identified_clusters = identified_clusters.swap_dims({"time": "cloud_id"})

# attrs = cloud_composite["radius"].attrs.copy()
# attrs.update({"units": "µm"})
# cloud_composite["radius"] = cloud_composite["radius"]
# cloud_composite["radius_micro"] = 1e6 * cloud_composite["radius"]
# cloud_composite["radius"].attrs = attrs

cloud_composite["radius2D"] = cloud_composite["radius"].expand_dims(time=cloud_composite["time"])
cloud_composite = cloud_composite.transpose("radius", ...)


# convert lwc and MSD to g m-3
# attrs = cloud_composite["liquid_water_content"].attrs.copy()
# attrs.update({"unit": "g m^{-3}"})
# cloud_composite['liquid_water_content'] = cloud_composite['liquid_water_content']
# cloud_composite['liquid_water_content'].attrs = attrs

attrs = cloud_composite["mass_size_distribution"].attrs.copy()
attrs.update({"unit": "g m^{-3} m^{-1}"})
cloud_composite["mass_size_distribution"] = cloud_composite["mass_size_distribution"] * 1e3
cloud_composite["mass_size_distribution"].attrs = attrs

cloud_composite = cloud_composite.sel(radius=slice(10e-6, None))

identified_clusters = identified_clusters.where(
    (
        (identified_clusters.duration.dt.seconds >= 3)
        & (identified_clusters.altitude < 1200)
        & (identified_clusters.altitude > 500)
    ),
    drop=True,
)
# identified_clusters = identified_clusters.isel(cloud_id=slice(0, 20))

In [ ]:
cleo_dataset = data_loading.CleoDataset(
    data_dir=CLEO_data_dir,
    microphysics=("null_microphysics",),
)
# get physicsal height cleo output data
ds_cleo, ds_cleo_sem = cleo_dataset()


# convert liquid water content to g m-3
attrs = ds_cleo["liquid_water_content"].attrs.copy()
attrs.update({"units": "g m^{-3}"})
attrs.update({"long_name": "Rain Water Content"})

ds_cleo["liquid_water_content"] = ds_cleo["liquid_water_content"] * 1e3
ds_cleo["liquid_water_content"].attrs = attrs

null_microphysics


In [ ]:
# load valid cloud ids
yaml = YAML(typ="safe")  # default, if not specfied, is 'rt' (round-trip)
d = yaml.load(CLEO_data_dir / Path("valid_cloud_ids.yaml"))
valid_cloud_ids = d["valid_cloud_ids"]


identified_clusters = identified_clusters.sel(cloud_id=valid_cloud_ids)
ds_cleo = ds_cleo.sel(cloud_id=valid_cloud_ids)

# Obtain Fit Parameters from files

In [ ]:
ds_parameters = xr.open_dataset(input_data_dir / "particle_size_distribution_parameters.nc")
ds_parameters_linear = xr.open_dataset(
    input_data_dir / "particle_size_distribution_parameters_linear_space.nc"
)

In [ ]:
radius = np.geomspace(50e-6, 3e-3, 100)

# radius = np.array([0, 1, 2, 4, 6, 8, 10, 15, 20, 25])

# radius = ds_cleo['radius_bins'].values * 1e-6

t_test = xr.DataArray(
    radius,
    dims="radius",
    coords={"radius": radius},
)
t_test = t_test.expand_dims(cloud_id=ds_cleo["cloud_id"])
w_test = (t_test["radius"].shift(radius=-1) - t_test["radius"].shift(radius=1)) / 2
w_test = w_test.interpolate_na("radius", method="linear", fill_value="extrapolate")

# noise = 0.1
# t_test = t_test + noise * np.random.randn(*t_test.shape) * t_test

In [ ]:
# fit the double log-normal distribution
ds_fitted: xr.DataArray = smodels.double_ln_normal_distribution(
    t=t_test,
    mu1=ds_parameters["mu1"],
    mu2=ds_parameters["mu2"],
    sigma1=ds_parameters["sigma1"],
    sigma2=ds_parameters["sigma2"],
    scale_factor1=ds_parameters["scale_factor1"],
    scale_factor2=ds_parameters["scale_factor2"],
)

fitted_psd = ds_fitted * w_test
fitted_msd = msd_from_psd_dataarray(ds_fitted * w_test)

In [ ]:
# fit the double log-normal distribution
ds_fitted_linear: xr.DataArray = smodels.double_log_normal_distribution_all(
    x=t_test,
    mu1=ds_parameters_linear["geometric_mean1"],
    mu2=ds_parameters_linear["geometric_mean2"],
    sigma1=ds_parameters_linear["geometric_std_dev1"],
    sigma2=ds_parameters_linear["geometric_std_dev2"],
    scale1=ds_parameters_linear["scale_factor1"],
    scale2=ds_parameters_linear["scale_factor2"],
    parameter_space="geometric",
    x_space="linear",
)

fitted_linear_psd = ds_fitted_linear * w_test
fitted_linear_msd = msd_from_psd_dataarray(ds_fitted_linear * w_test)

In [ ]:
ds_potential_temperature_parameters = xr.open_dataset(
    input_data_dir / "potential_temperature_parameters.nc"
)
ds_pressure_parameters = xr.open_dataset(input_data_dir / "pressure_parameters.nc")
ds_relative_humidity_parameters = xr.open_dataset(input_data_dir / "relative_humidity_parameters.nc")

In [ ]:
da_potential_temperature: xr.DataArray = smodels.split_linear_func(
    x=dropsonde["altitude"],
    f_0=ds_potential_temperature_parameters["f_0"],
    slope_1=ds_potential_temperature_parameters["slope_1"],
    slope_2=ds_potential_temperature_parameters["slope_2"],
    x_split=ds_potential_temperature_parameters["x_split"],
)
da_potential_temperature = da_potential_temperature.sel(altitude=slice(0, 1200))

da_relative_humidity: xr.DataArray = smodels.split_linear_func(
    x=dropsonde["altitude"],
    f_0=ds_relative_humidity_parameters["f_0"],
    slope_1=ds_relative_humidity_parameters["slope_1"],
    slope_2=ds_relative_humidity_parameters["slope_2"],
    x_split=ds_relative_humidity_parameters["x_split"],
)
da_relative_humidity = da_relative_humidity.sel(altitude=slice(0, 1200))

da_pressure: xr.DataArray = smodels.linear_func(
    x=dropsonde["altitude"],
    slope=ds_pressure_parameters["slope"],
    f_0=ds_pressure_parameters["f_0"],
)
da_pressure = da_pressure.sel(altitude=slice(0, 1200))

<!-- # Random cloud example -->

In [ ]:
cleo_dataset = data_loading.CleoDataset(
    data_dir=CLEO_data_dir,
    microphysics=("null_microphysics",),
)
# get physicsal height cleo output data
ds_cleo, ds_cleo_sem = cleo_dataset()


# convert liquid water content to g m-3
attrs = ds_cleo["liquid_water_content"].attrs.copy()
attrs.update({"units": "g m^{-3}"})
attrs.update({"long_name": "Rain Water Content"})

ds_cleo["liquid_water_content"] = ds_cleo["liquid_water_content"] * 1e3
ds_cleo["liquid_water_content"].attrs = attrs

null_microphysics


In [ ]:
# load valid cloud ids
yaml = YAML(typ="safe")  # default, if not specfied, is 'rt' (round-trip)
d = yaml.load(CLEO_data_dir / Path("valid_cloud_ids.yaml"))
valid_cloud_ids = d["valid_cloud_ids"]


identified_clusters = identified_clusters.sel(cloud_id=valid_cloud_ids)
ds_cleo = ds_cleo.sel(cloud_id=valid_cloud_ids)
ds_cleo_sem = ds_cleo_sem.sel(cloud_id=valid_cloud_ids)

# Obtain observation RWC, LWC and NBC

In [ ]:
list_lwc = []
list_lwc_sem = []
list_lwc_50um = []
list_lwc_50um_sem = []

list_nbc = []
list_nbc_sem = []
list_nbc_50um = []
list_nbc_50um_sem = []
i = 0
N = len(identified_clusters["cloud_id"])
for cloud_id in identified_clusters["cloud_id"]:
    # i += 1
    # print(f"Processing cloud {i}/{N}")
    cc = match_clouds_and_cloudcomposite(
        ds_clouds=identified_clusters.sel(cloud_id=cloud_id),
        ds_cloudcomposite=cloud_composite,
    )
    cc = cc.expand_dims(cloud_id=[int(cloud_id.values)])

    lwc = cc["liquid_water_content"]

    lwc_mean, lwc_sem = mean_and_stderror_of_mean(lwc, dims=("time",))
    list_lwc.append(lwc_mean)
    list_lwc_sem.append(lwc_sem)

    lwc_above_50um = (
        (cc["mass_size_distribution"] * cc["bin_width"]).sel(radius=slice(50e-6, None)).sum("radius")
    )
    lwc_above_50um_mean, lwc_above_50um_sem = mean_and_stderror_of_mean(lwc_above_50um, dims=("time",))
    list_lwc_50um.append(lwc_above_50um_mean)
    list_lwc_50um_sem.append(lwc_above_50um_sem)

    nbc = cc["particle_size_distribution"] * cc["bin_width"]  # .sum("radius")

    nbc_mean, nbc_sem = mean_and_stderror_of_mean(nbc, dims=("time",))
    list_nbc.append(nbc_mean)
    list_nbc_sem.append(nbc_sem)

    nbc_above_50um = (
        (cc["particle_size_distribution"] * cc["bin_width"]).sel(radius=slice(50e-6, None)).sum("radius")
    )
    nbc_above_50um_mean, nbc_above_50um_sem = mean_and_stderror_of_mean(nbc_above_50um, dims=("time",))
    list_nbc_50um.append(nbc_above_50um_mean)
    list_nbc_50um_sem.append(nbc_above_50um_sem)


da_lwc = xr.concat(
    list_lwc,
    dim="cloud_id",
)
da_lwc.attrs = dict(
    long_name="Liquid water content",
    units="g m^{-3}",
)

da_lwc_sem = xr.concat(
    list_lwc_sem,
    dim="cloud_id",
)
da_lwc_sem.attrs = dict(
    long_name="Standard error of the mean of the liquid water content",
    units="g m^{-3}",
)

da_lwc_50um = xr.concat(
    list_lwc_50um,
    dim="cloud_id",
)
da_lwc_50um.attrs = dict(
    long_name="Rain Water Content",
    units="g m^{-3}",
)

da_lwc_50um_sem = xr.concat(
    list_lwc_50um_sem,
    dim="cloud_id",
)
da_lwc_50um_sem.attrs = dict(
    long_name="Standard error of the mean of the Rain Water Content",
    units="g m^{-3}",
)

da_nbc = xr.concat(
    list_nbc,
    dim="cloud_id",
)
da_nbc.attrs = dict(
    long_name="Number concentration",
    units="m^{-3}",
)

da_nbc_sem = xr.concat(
    list_nbc_sem,
    dim="cloud_id",
)
da_nbc_sem.attrs = dict(
    long_name="Standard error of the mean of the number concentration",
    units="m^{-3}",
)

da_nbc_50um = xr.concat(
    list_nbc_50um,
    dim="cloud_id",
)
da_nbc_50um.attrs = dict(
    long_name="Number concentration above 50 µm",
    units="m^{-3}",
)

da_nbc_50um_sem = xr.concat(
    list_nbc_50um_sem,
    dim="cloud_id",
)
da_nbc_50um_sem.attrs = dict(
    long_name="Standard error of the mean of the number concentration above 50 µm",
    units="m^{-3}",
)


ds_observations_backup = xr.Dataset(
    dict(
        liquid_water_content=da_lwc,
        liquid_water_content_sem=da_lwc_sem,
        liquid_water_content_50um=da_lwc_50um,
        liquid_water_content_50um_sem=da_lwc_50um_sem,
        number_concentration=da_nbc,
        number_concentration_sem=da_nbc_sem,
        number_concentration_50um=da_nbc_50um,
        number_concentration_50um_sem=da_nbc_50um_sem,
    )
)

In [ ]:
ds_observations = ds_observations_backup
ds_observations = ds_observations.sel(cloud_id=valid_cloud_ids)

In [ ]:
# ds_observations_backup["liquid_water_content"].plot()
# ds_observations_backup["liquid_water_content_50um"].plot()

# Visualize 

In [ ]:
def plot_4_all(cloud_id, pressure_reference):

    fig, axs = plt.subplot_mosaic(
        [
            [
                "ax_pt",
                "ax_rh",
            ],
            [
                "ax_psd",
                "ax_lwc",
            ],
        ],
        figsize=large_square_fig_size,
        layout="constrained",
    )

    ax_psd: plt.Axes = axs["ax_psd"]
    ax_lwc: plt.Axes = axs["ax_lwc"]
    ax_pt: plt.Axes = axs["ax_pt"]
    ax_rh: plt.Axes = axs["ax_rh"]

    # PARTICLE SIZE DISTRIBUTION

    ds_observed_cloud = match_clouds_and_cloudcomposite(
        ds_clouds=identified_clusters.sel(cloud_id=cloud_id),
        ds_cloudcomposite=cloud_composite,
    )

    ds_dropsonde_cloud = match_clouds_and_dropsondes(
        ds_clouds=identified_clusters.sel(cloud_id=cloud_id),
        ds_sonde=dropsonde,
        ds_distance=ds_distances,
        max_temporal_distance=np.timedelta64("3", "h"),
        max_spatial_distance=100,
    )
    ds_dropsonde_cloud = ds_dropsonde_cloud.sel(altitude=slice(0, 1200))

    x_observed = ds_observed_cloud["radius"] * 1e6
    y_observed = ds_observed_cloud["particle_size_distribution"]

    ds_cleo_cloud = ds_cleo.sel(gridbox=ds_cleo["max_gridbox"])
    ds_cleo_cloud = ds_cleo_cloud.sel(cloud_id=cloud_id)
    x_cleo = ds_cleo_cloud["radius_bins"]

    w_cleo = (x_cleo.shift(radius_bins=-1) - x_cleo.shift(radius_bins=1)) / 2
    w_cleo = w_cleo.interpolate_na("radius_bins", method="linear", fill_value="extrapolate")
    w_cleo = w_cleo * 1e-6  # convert µm to m

    y_cleo = ds_cleo_cloud["xi_temporal_mean"] / w_cleo / ds_cleo_cloud["gridbox_volume"]
    y_cleo = y_cleo.isel(microphysics=0)
    y_cleo.attrs.update(
        units="m^{-3} m^{-1}",
        long_name="Number concentration",
    )

    ds_fitted_cloud = ds_fitted_linear.sel(cloud_id=cloud_id)
    y_fitted = ds_fitted_cloud
    x_fitted = ds_fitted_cloud["radius"] * 1e6
    # ax_psd.set_title("Observed PSD")

    # plot observed and fitted PSD
    ax_psd.plot(
        x_observed,
        y_observed,
        linestyle="",
        marker=".",
        markersize=2,
        color=[0.5, 0.5, 0.5],
        alpha=0.3,
    )
    ax_psd.plot(
        x_observed,
        y_observed.mean("time"),
        linestyle="",
        linewidth=1,
        marker=".",
        markersize=4,
        color=[0.3, 0.3, 0.3],
        alpha=0.75,
        label="Mean. Obs.",
    )

    # ax_psd.plot(
    #     x_cleo,
    #     y_cleo.T,
    #     linestyle="--",
    #     color=[0.1,0.1,0.1],
    #     lw=2,
    #     label="CLEO",
    # )

    ax_psd.plot(
        x_fitted,
        y_fitted.T,
        linestyle="-",
        color=[0.1, 0.1, 0.1],
        lw=2,
        label="Fit",
    )

    ax_psd.set_xscale("log")
    ax_psd.set_yscale("symlog", linthresh=0.5, linscale=0.3)
    ax_psd.set_ylim(-0.5, 1e13)
    # ax_psd.set_ylim(0, 20e6)
    # ax_psd.set_xlim(0, 3000)
    # ax_psd.set_ylim(0, 5e6)
    ax_psd.set_xlabel(r"Radius $[\mu m]$")
    ax_psd.set_ylabel(r"PSD $[m^{-3} m^{-1}]$")

    # DROPSONDE

    # ----------------------
    # Liquid Water Content
    # ----------------------

    x = ds_observations["liquid_water_content_50um"]
    xerr = ds_observations["liquid_water_content_50um_sem"]
    y = ds_cleo["cloud_liquid_water_content"].sel(microphysics="null_microphysics")
    yerr = ds_cleo_sem["cloud_liquid_water_content"].sel(microphysics="null_microphysics")

    y.attrs.update(long_name="Model " + y.attrs["long_name"])

    # indices = (x != 0) & (y != 0)

    # x, xerr, y, yerr = x[indices], xerr[indices], y[indices], yerr[indices]

    corr = xr.corr(x, y, dim="cloud_id")
    corr_loglog = xr.corr(np.log(x + 1e-28), np.log(y + 1e-28), dim="cloud_id")

    ax_lwc.scatter(
        x=x,
        y=y,
        marker=".",
        color="k",
        alpha=0.75,
        # label = f"Pearson correlation coefficient: {corr.values:.2f}"
    )

    ax_lwc.errorbar(
        x=x,
        y=y,
        xerr=xerr,
        yerr=yerr,
        fmt=".",
        color="k",
        alpha=0.1,
        # label = f"Pearson correlation coefficient: {corr.values:.2f}"
    )

    xlim = ax_lwc.get_xlim()
    ylim = ax_lwc.get_ylim()
    lim = 0, max(xlim[1], ylim[1])

    ax_lwc.set_xlim(lim)
    ax_lwc.set_ylim(lim)

    plot_one_one(ax_lwc, color="grey", linestyle="-")
    ax_lwc.set_xlabel(label_from_attrs(x, name_width=25))
    ax_lwc.set_ylabel(label_from_attrs(y, name_width=20))
    # ax.legend(loc="upper left")

    ax_lwc.annotate(
        text=f"$R = {corr.values:.2f}$\n$R_{{log}} = {corr_loglog.values:.2f}$",
        xy=(0.65, 0.1),
        xycoords="axes fraction",
        # fontsize=10,
    )

    # ----------------------
    # POTENTIAL TEMPERATURE
    # ----------------------

    y_observed = ds_dropsonde_cloud["altitude"]
    x_observed = ds_dropsonde_cloud["potential_temperature"].transpose(..., "time")

    x_cleo = potential_temperature_from_temperature_pressure(
        air_temperature=ds_cleo["air_temperature"].sel(cloud_id=cloud_id),
        pressure=ds_cleo["pressure"].sel(cloud_id=cloud_id),
        pressure_reference=pressure_reference,
    )
    y_cleo = ds_cleo["gridbox_coord3"].sel(cloud_id=cloud_id)

    x_fitted = da_potential_temperature.sel(cloud_id=cloud_id)
    y_fitted = da_pressure["altitude"]

    ax_pt.plot(
        x_observed,
        y_observed,
        linestyle="-",
        color=default_colors[0],
        alpha=0.3,
    )
    ax_pt.plot(
        x_observed.mean("time"),
        y_observed,
        linestyle="-",
        color=default_dark_colors[0],
        alpha=0.75,
        lw=1,
        label="Mean. Obs.",
    )
    # ax_pt.plot(
    #     x_cleo.T,
    #     y_cleo.T,
    #     linestyle="--",
    #     color=default_dark_colors[0],
    #     lw=2,
    #     label="CLEO",
    # )

    ax_pt.plot(
        x_fitted.T,
        y_fitted.T,
        linestyle="-",
        color=default_dark_colors[0],
        lw=2,
        label="Fit",
    )

    ax_pt.set_xlabel("Pot. Temp. $[K]$")
    ax_pt.set_ylabel("Altitude $[m]$")

    # ----------------------
    # RELATIVE HUMIDITY
    # ----------------------

    y_observed = ds_dropsonde_cloud["altitude"]
    x_observed = ds_dropsonde_cloud["relative_humidity"].transpose(..., "time")

    x_cleo = relative_humidity_from_tps(
        temperature=ds_cleo["air_temperature"].sel(cloud_id=cloud_id),
        pressure=ds_cleo["pressure"].sel(cloud_id=cloud_id),
        specific_humidity=ds_cleo["specific_mass_vapour"].sel(cloud_id=cloud_id),
    )
    y_cleo = ds_cleo["gridbox_coord3"].sel(cloud_id=cloud_id)

    x_fitted = da_relative_humidity.sel(cloud_id=cloud_id)
    y_fitted = da_relative_humidity["altitude"]

    ax_rh.plot(
        x_observed,
        y_observed,
        linestyle="-",
        color=default_colors[1],
        alpha=0.3,
    )
    ax_rh.plot(
        x_observed.mean("time"),
        y_observed,
        linestyle="-",
        color=default_dark_colors[1],
        alpha=0.75,
        lw=1,
        label="Mean. Obs.",
    )
    # ax_rh.plot(
    #     x_cleo.T,
    #     y_cleo.T,
    #     linestyle="--",
    #     color=default_dark_colors[1],
    #     lw=2,
    #     label="CLEO",
    # )
    ax_rh.plot(
        x_fitted.T,
        y_fitted.T,
        linestyle="-",
        color=default_dark_colors[1],
        lw=2,
        label="Fit",
    )

    yticks = ax_pt.get_yticks()
    # ax_pt.set_yticks(yticks, np.full_like(yticks, "", dtype=str))
    ax_pt.set_xticks(np.arange(296, 302, 2))

    ax_rh.set_xlabel(r"Rel. Hum. $[\%]$")
    # ax_rh.set_yticks(yticks, np.full_like(yticks, "", dtype=str))

    for _ax in [ax_pt, ax_rh]:
        _ax.axhline(
            ds_relative_humidity_parameters["x_split"].sel(cloud_id=cloud_id).data,
            color="red",
            linestyle=":",
        )
        _ax.axhline(ds_observed_cloud["altitude"].mean("time").data, color="k", linestyle=":")
        _ax.set_yticks(np.arange(0, 1250, 400))
        _ax.set_ylim(0, 1200)
        # _ax.tick_params(axis="x", rotation=45)

    ax_psd.legend(loc="lower left")

    ax_psd.set_title(f"Measurements {len(ds_observed_cloud['time'].data)}")
    ax_pt.set_title(f"Measurements {len(ds_dropsonde_cloud['time'].data)}")
    ax_rh.set_title(f"Measurements {len(ds_dropsonde_cloud['time'].data)}")

    # # remove all labels and spines
    # for _ax in axs:
    #     _ax.spines["top"].set_visible(False)
    #     _ax.spines["right"].set_visible(False)
    #     _ax.spines["left"].set_visible(False)
    #     _ax.spines["bottom"].set_visible(False)

    #     _ax.tick_params(
    #         axis="both",
    #         which="both",
    #         bottom=False,
    #         top=False,
    #         left=False,
    #         right=False,
    #         labelbottom=False,
    #         labelleft=False,
    #     )
    #     _ax.set_xlabel("")
    #     _ax.set_ylabel("")
    #     _ax.set_title("")

    # fig.tight_layout()
    # ax_pt.set_ylim(0, None)
    fig.suptitle(
        f"Cloud ID {cloud_id} | Pressure Ref. {pressure_reference/100:.2f} hPa | Time {ds_observed_cloud['time'].data[0].astype('datetime64[m]')}"
    )
    fig.tight_layout()
    return fig, axs

In [ ]:
cloud_id = 396
fig, axs = plot_4_all(cloud_id=cloud_id, pressure_reference=100000)
add_subplotlabel(
    axs=axs.values(),
    location="upper left",
)

axs["ax_psd"].set_yticks([0, 1e0, 1e3, 1e6, 1e9, 1e12])
lwc_ticks = np.arange(0, 0.8, 0.2)
axs["ax_lwc"].set_yticks(lwc_ticks)
axs["ax_lwc"].set_xticks(lwc_ticks)
axs["ax_lwc"].set_ylabel("Fitted Rain Water\nContent " + r"$[g m^{-3}]$")
axs["ax_lwc"].set_xlabel("Observed Rain Water\nContent " + r"$[g m^{-3}]$")

axs["ax_lwc"].scatter(
    x=ds_observations["liquid_water_content_50um"].sel(cloud_id=cloud_id),
    y=ds_cleo["cloud_liquid_water_content"].sel(microphysics="null_microphysics").sel(cloud_id=cloud_id),
    marker=".",
    color="r",
    alpha=1,
    zorder=10,
)

for key in axs:
    try:
        axs[key].get_legend().remove()
    except AttributeError:
        pass

for _ax in [axs["ax_pt"], axs["ax_rh"]]:
    xlim = _ax.get_xlim()
    xy = (xlim[0], 200)
    width = xlim[1] - xlim[0]
    height = 500 - 200
    rect = mpatches.Rectangle(
        xy=xy,
        width=width,
        height=height,
        linewidth=0,
        edgecolor="None",
        facecolor=[0.5, 0.5, 0.5, 0.1],
    )

    # Add the patch to the Axes
    _ax.add_patch(rect)

fig.suptitle("")
for _ax in axs.values():
    _ax.set_title("")

fig.tight_layout()
save_figure(
    fig=fig,
    filepath=fig_dir / f"compare_observations_cloud_{cloud_id}",
)

/tmp/ipykernel_3519830/3518939793.py:313: UserWarning: The figure layout has changed to tight
  fig.tight_layout()


In [ ]:
np.random.seed(32)
cloud_id = np.random.choice(valid_cloud_ids, 1)[0].item()
fig, axs = plot_4_all(cloud_id=cloud_id, pressure_reference=100000)
add_subplotlabel(
    axs=axs.values(),
    location="upper left",
)

axs["ax_psd"].set_yticks([0, 1e0, 1e3, 1e6, 1e9, 1e12])
lwc_ticks = np.arange(0, 0.8, 0.2)
axs["ax_lwc"].set_yticks(lwc_ticks)
axs["ax_lwc"].set_xticks(lwc_ticks)
axs["ax_lwc"].set_ylabel("Fitted Rain Water\nContent " + r"$[g m^{-3}]$")
axs["ax_lwc"].set_xlabel("Observed Rain Water\nContent " + r"$[g m^{-3}]$")

axs["ax_lwc"].scatter(
    x=ds_observations["liquid_water_content_50um"].sel(cloud_id=cloud_id),
    y=ds_cleo["cloud_liquid_water_content"].sel(microphysics="null_microphysics").sel(cloud_id=cloud_id),
    marker=".",
    color="r",
    alpha=1,
    zorder=10,
)

for key in axs:
    try:
        axs[key].get_legend().remove()
    except AttributeError:
        pass

for _ax in [axs["ax_pt"], axs["ax_rh"]]:
    xlim = _ax.get_xlim()
    xy = (xlim[0], 200)
    width = xlim[1] - xlim[0]
    height = 500 - 200
    rect = mpatches.Rectangle(
        xy=xy,
        width=width,
        height=height,
        linewidth=0,
        edgecolor="None",
        facecolor=[0.5, 0.5, 0.5, 0.1],
    )

    # Add the patch to the Axes
    _ax.add_patch(rect)

fig.suptitle("")
for _ax in axs.values():
    _ax.set_title("")

fig.tight_layout()
save_figure(
    fig=fig,
    filepath=fig_dir / f"compare_observations_cloud_{cloud_id}",
)

/tmp/ipykernel_3519830/3518939793.py:313: UserWarning: The figure layout has changed to tight
  fig.tight_layout()
